# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [11]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import numpy as np
import requests
import json

# Import API key
from api_keys import geoapify_key
print (geoapify_key)

070d4f77c6f74ceb87d17b58d1ce505d


In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Crane,31.3974,-102.3501,66.20,22,8,13.96,US,1714137451
1,1,Thal,35.4791,72.2439,48.60,74,100,3.58,PK,1714137453
2,2,Weno,7.4515,151.8468,84.15,94,75,9.22,FM,1714137456
3,3,Puerto Ayora,-0.7393,-90.3518,77.56,96,71,1.01,EC,1714137458
4,4,Buala,-8.1450,159.5921,81.43,82,100,3.24,SB,1714137461


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 0.5,
    color = "City")

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_df_copy = city_data_df.copy()
city_data_df_copy = city_data_df_copy.dropna()
city_data_df_copy
vaca_spot = []
# Drop any rows with low and high temps
for i, row in city_data_df_copy.iterrows():
    if row['Max Temp']>60 and row['Max Temp']<85 and row['Wind Speed']<5 and row['Humidity']<65:
        vaca_spot.append(row)

# Display sample data
vaca_df = pd.DataFrame(vaca_spot)
vaca_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
7,7,Wāngjing,24.6000,94.0333,75.04,50,0,2.10,IN,1714137467
28,28,Shevchenkove,49.6959,37.1735,75.16,41,100,1.97,UA,1714137511
101,101,Mutuáli,-14.8706,37.0044,83.79,36,93,2.13,MZ,1714137656
140,140,Chingirlau,51.0950,54.0814,75.07,43,74,3.22,KZ,1714137732
149,149,San Blas,21.5167,-105.2667,66.09,53,100,4.52,MX,1714137749
155,155,Uttarkāshi,30.7333,78.4500,68.85,24,0,2.68,IN,1714137759
177,177,Vallenar,-28.5708,-70.7581,70.59,40,100,0.36,CL,1714137800
185,185,Changping,40.2186,116.2197,65.84,46,95,2.91,CN,1714137816
210,210,Haffouz,35.6324,9.6762,70.83,42,100,3.31,TN,1714137865
211,211,Acajutla,13.5928,-89.8275,83.41,55,0,3.94,SV,1714137866


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = vaca_df.copy()
hotel_df = hotel_df.drop(['City_ID','Max Temp','Cloudiness','Wind Speed','Date'], axis = 1)
hotel_df
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"]=""

# Display sample data
hotel_df

,City,Lat,Lng,Humidity,Country,Hotel Name
7,Wāngjing,24.6000,94.0333,50,IN,
28,Shevchenkove,49.6959,37.1735,41,UA,
101,Mutuáli,-14.8706,37.0044,36,MZ,
140,Chingirlau,51.0950,54.0814,43,KZ,
149,San Blas,21.5167,-105.2667,53,MX,
155,Uttarkāshi,30.7333,78.4500,24,IN,
177,Vallenar,-28.5708,-70.7581,40,CL,
185,Changping,40.2186,116.2197,46,CN,
210,Haffouz,35.6324,9.6762,42,TN,
211,Acajutla,13.5928,-89.8275,55,SV,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [15]:
# Set parameters to search for a hotel
# url = "https://api.geoapify.com/v2/places?categories=accommodation.hotel&filter=circle:-94.685738,38.607803,5000&bias=proximity:-94.685738,38.607803&limit=20&apiKey=a7efb108fbfc418484d268e9f5dedd0"
# Set parameters to search for a hotel
radius = 10000
# params = {"filter": filter,
#           "radius":radius,
#           "apiKey": weather_api_key,
#           "categories":"accommodation.hotel"
#          }
# base_url = "https://api.geoapify.com/v2/places?"

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    
     # get latitude, longitude from the DataFrame
    long = row["Lng"]
    lat = row["Lat"]
  
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    filter = f"circle:{long},{lat},{radius}"
    bias = f"proximity:{long},{lat}"
    
    # Set base URL
    base_url = f"https://api.geoapify.com/v2/places?categories=accommodation.hotel&filter={filter}&bias={bias}&apiKey={geoapify_key}"
    name_address = requests.get(base_url)
    print(base_url)
    # Make an API request using the params dictionary
    # name_address = # YOUR CODE HERE
    
    #Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
https://api.geoapify.com/v2/places?categories=accommodation.hotel&filter=circle:94.0333,24.6,10000&bias=proximity:94.0333,24.6&apiKey=070d4f77c6f74ceb87d17b58d1ce505d
Wāngjing - nearest hotel: No hotel found
https://api.geoapify.com/v2/places?categories=accommodation.hotel&filter=circle:37.1735,49.6959,10000&bias=proximity:37.1735,49.6959&apiKey=070d4f77c6f74ceb87d17b58d1ce505d
Shevchenkove - nearest hotel: No hotel found
https://api.geoapify.com/v2/places?categories=accommodation.hotel&filter=circle:37.0044,-14.8706,10000&bias=proximity:37.0044,-14.8706&apiKey=070d4f77c6f74ceb87d17b58d1ce505d
Mutuáli - nearest hotel: No hotel found
https://api.geoapify.com/v2/places?categories=accommodation.hotel&filter=circle:54.0814,51.095,10000&bias=proximity:54.0814,51.095&apiKey=070d4f77c6f74ceb87d17b58d1ce505d
Chingirlau - nearest hotel: No hotel found
https://api.geoapify.com/v2/places?categories=accommodation.hotel&filter=circle:-105.2667,21.5167,10000&bias=proximity:-105

,City,Lat,Lng,Humidity,Country,Hotel Name
7,Wāngjing,24.6000,94.0333,50,IN,No hotel found
28,Shevchenkove,49.6959,37.1735,41,UA,No hotel found
101,Mutuáli,-14.8706,37.0044,36,MZ,No hotel found
140,Chingirlau,51.0950,54.0814,43,KZ,No hotel found
149,San Blas,21.5167,-105.2667,53,MX,Suites San Blas
155,Uttarkāshi,30.7333,78.4500,24,IN,Shivalinga
177,Vallenar,-28.5708,-70.7581,40,CL,Hotel del Marqués
185,Changping,40.2186,116.2197,46,CN,速8酒店
210,Haffouz,35.6324,9.6762,42,TN,No hotel found
211,Acajutla,13.5928,-89.8275,55,SV,Hotel Lara


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [18]:
%%capture --no-display

# Configure the map plot
Vacation_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    
    scale = 2,
    color = "City",
    hover_cols = ["Hotel Name", "City"])

# Display the map
Vacation_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name)